In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


/Users/kevin/.pyenv/versions/3.10.2/envs/python_bytes_generator/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("arvkevi/nba_pbp_distilgpt2")

model = AutoModelForCausalLM.from_pretrained("arvkevi/nba_pbp_distilgpt2", pad_token_id=tokenizer.eos_token_id)


In [42]:
full_pbp = ""

prompt = "1 11:45 0-0 K. Looney misses 2-pt layup from 4 ft (block by R. Williams)"
# for _ in range(10):
while "End of Game" not in full_pbp:
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    print(prompt)
    outputs = model.generate(
        input_ids,
        do_sample=True,
        max_length=1024,
        temperature=0.95,
        top_k=0,
        top_p=0.90,
        repitition_penalty=1.0,
    )

    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]
    # the last few tokens might be an imcomplete play
    generated_text_remove_last_line = generated_text.splitlines()[:-1]
    prompt = "\n".join(generated_text_remove_last_line[-3:])
    generated_text_remove_last_line = "\n".join(generated_text_remove_last_line)

    full_pbp+=generated_text_remove_last_line

1 11:45 0-0 K. Looney misses 2-pt layup from 4 ft (block by R. Williams)
1 7:00 2-0 R. Langford misses 2-pt layup from 3 ft
1 7:00 2-0 Offensive rebound by Team
1 7:00 2-0 R. Langford misses 3-pt jump shot from 24 ft
1 3:43 5-2 Offensive rebound by D. Schröder
1 3:41 5-2 M. Smart misses 2-pt layup from 6 ft
1 3:40 5-2 Offensive rebound by D. Schröder
1 1:02 7-4 A. Horford misses 2-pt jump shot from 16 ft
1 1:01 7-4 Defensive rebound by D. Schröder
1 1:00 7-4 Turnover by M. Smart (lost ball; steal by J. Richardson)
2 9:14 10-9 Turnover by M. Smart (out of bounds lost ball)
2 9:07 10-9 G. Williams misses 3-pt jump shot from 24 ft
2 9:06 10-9 Defensive rebound by J. Richardson
2 6:19 17-10 J. Brown misses free throw 1 of 2
2 6:14 17-10 Defensive rebound by A. Horford
2 6:14 17-10 D. Schröder misses 2-pt layup from 4 ft (block by A. Horford)
2 3:22 19-11 J. Brown misses 2-pt jump shot from 6 ft
2 3:20 19-11 Defensive rebound by D. Schröder
2 3:21 19-11 Personal foul by M. Smart (drawn by C

In [ ]:
with open("BOS_GSW_distilgpt2_pbp.txt", "w") as f:
    f.write(full_pbp)